In [1]:
# dependencies
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
%matplotlib inline
# import dependencies
from getpass import getpass
import sqlalchemy
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, text
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf

In [2]:
# SQL Alchemy connection from RDS
secret = getpass('Enter the secret value: ')

args ={
    'host':"ogdataset.c11hekhsylui.us-west-1.rds.amazonaws.com",
    'port':'5432',
    'database':"og_dataset",
    'user':"attritionProject",
    'password':secret
}
engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{database}".format(**args))
connection = engine.connect()

Enter the secret value: ········


/Users/stephen/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# read in table from database
attrition_df = pd.read_sql('SELECT * FROM new_encoded_data', connection)
attrition_df

,Age,Attrition,Business Travel,Department,Distance from Home,Education Level,Education Field,Environment Satisfaction,Gender,Job Involvement,...,Performance Rating,Relationship Satisfaction,Stock Option Level,Total Working Years,Training Times Last Year,Work Life Balance,Years at Company,Years In Current Role,Years Since Last Promotion,Years With Current Manager
0,41,1,2,2,1,2,1,2,0,3,...,3,1,0,8,0,1,6,4,0,5
1,49,0,1,1,8,1,1,3,1,2,...,4,4,1,10,3,3,10,7,1,7
2,37,1,2,1,2,2,4,4,1,2,...,3,2,0,7,3,3,0,0,0,0
3,33,0,1,1,3,4,1,4,0,3,...,3,3,0,8,3,3,8,7,3,0
4,27,0,2,1,2,1,3,1,1,3,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,0,1,1,23,2,3,3,1,4,...,3,3,1,17,3,3,5,2,0,3
1466,39,0,2,1,6,1,3,4,1,2,...,3,1,1,9,5,3,7,7,1,7
1467,27,0,2,1,4,3,1,2,1,4,...,4,2,1,6,0,3,6,2,0,3
1468,49,0,1,2,2,3,3,4,1,2,...,3,4,0,17,3,2,9,6,0,8


In [4]:
# check version number
import imblearn
print(imblearn.__version__)

0.7.0


In [5]:
attrition_df.drop(['Total Working Years','Years In Current Role','Job Level', 'Years With Current Manager'],axis=1, inplace=True)


In [6]:
# Split the preprocessed data into a training and testing dataset
# Split our preprocessed data into our features and target arrays
X = attrition_df.drop("Attrition",1)
y = attrition_df["Attrition"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, test_size= 0.20, train_size= 0.80, stratify =y)

/Users/stephen/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

In [8]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler# define oversampling strategy
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_train, y_train)
# summarize class distribution
print(Counter(y_over))

Counter({0: 986, 1: 986})


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_over)

# Scale the data
X_train_scaled = X_scaler.transform(X_over)
X_test_scaled = X_scaler.transform(X_test)

In [10]:


# Create scaler instance
#X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
#X_scaler.fit(X_train)

# Scale the data
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)



In [11]:
len(X_train_scaled)

1972

In [12]:
len(y_over)

1972

In [13]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 800
hidden_nodes_layer2 = 600

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_over, epochs=500)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/500


2022-06-08 14:48:13.128994: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


62/62 [==============================] - 0s 3ms/step - loss: 0.7253 - accuracy: 0.5669
Epoch 2/500
62/62 [==============================] - 0s 3ms/step - loss: 0.5941 - accuracy: 0.6897
Epoch 3/500
62/62 [==============================] - 0s 2ms/step - loss: 0.5652 - accuracy: 0.7125
Epoch 4/500
62/62 [==============================] - 0s 2ms/step - loss: 0.5727 - accuracy: 0.7064
Epoch 5/500
62/62 [==============================] - 0s 2ms/step - loss: 0.5384 - accuracy: 0.7348
Epoch 6/500
62/62 [==============================] - 0s 2ms/step - loss: 0.5387 - accuracy: 0.7394
Epoch 7/500
62/62 [==============================] - 0s 2ms/step - loss: 0.5379 - accuracy: 0.7404
Epoch 8/500
62/62 [==============================] - 0s 2ms/step - loss: 0.5273 - accuracy: 0.7444
Epoch 9/500
62/62 [==============================] - 0s 2ms/step - loss: 0.5265 - accuracy: 0.7378
Epoch 10/500
62/62 [==============================] - 0s 2ms/step - loss: 0.5232 - accuracy: 0.7571
Epoch 11/500
62/62 [=

62/62 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 165/500
62/62 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 166/500
62/62 [==============================] - 0s 2ms/step - loss: 9.6340e-04 - accuracy: 1.0000
Epoch 167/500
62/62 [==============================] - 0s 2ms/step - loss: 9.5329e-04 - accuracy: 1.0000
Epoch 168/500
62/62 [==============================] - 0s 2ms/step - loss: 8.7260e-04 - accuracy: 1.0000
Epoch 169/500
62/62 [==============================] - 0s 2ms/step - loss: 8.1651e-04 - accuracy: 1.0000
Epoch 170/500
62/62 [==============================] - 0s 2ms/step - loss: 7.7381e-04 - accuracy: 1.0000
Epoch 171/500
62/62 [==============================] - 0s 2ms/step - loss: 7.5145e-04 - accuracy: 1.0000
Epoch 172/500
62/62 [==============================] - 0s 2ms/step - loss: 6.9834e-04 - accuracy: 1.0000
Epoch 173/500
62/62 [==============================] - 0s 2ms/step - loss: 6.

62/62 [==============================] - 0s 2ms/step - loss: 1.6543e-06 - accuracy: 1.0000
Epoch 321/500
62/62 [==============================] - 0s 2ms/step - loss: 1.5882e-06 - accuracy: 1.0000
Epoch 322/500
62/62 [==============================] - 0s 2ms/step - loss: 1.5611e-06 - accuracy: 1.0000
Epoch 323/500
62/62 [==============================] - 0s 2ms/step - loss: 1.4786e-06 - accuracy: 1.0000
Epoch 324/500
62/62 [==============================] - 0s 2ms/step - loss: 1.4243e-06 - accuracy: 1.0000
Epoch 325/500
62/62 [==============================] - 0s 2ms/step - loss: 1.3881e-06 - accuracy: 1.0000
Epoch 326/500
62/62 [==============================] - 0s 2ms/step - loss: 1.3555e-06 - accuracy: 1.0000
Epoch 327/500
62/62 [==============================] - 0s 2ms/step - loss: 1.2899e-06 - accuracy: 1.0000
Epoch 328/500
62/62 [==============================] - 0s 2ms/step - loss: 1.2681e-06 - accuracy: 1.0000
Epoch 329/500
62/62 [==============================] - 0s 2ms/step - 

62/62 [==============================] - 0s 2ms/step - loss: 2.6704e-08 - accuracy: 1.0000
Epoch 477/500
62/62 [==============================] - 0s 2ms/step - loss: 2.6403e-08 - accuracy: 1.0000
Epoch 478/500
62/62 [==============================] - 0s 2ms/step - loss: 2.5886e-08 - accuracy: 1.0000
Epoch 479/500
62/62 [==============================] - 0s 2ms/step - loss: 2.5425e-08 - accuracy: 1.0000
Epoch 480/500
62/62 [==============================] - 0s 2ms/step - loss: 2.5178e-08 - accuracy: 1.0000
Epoch 481/500
62/62 [==============================] - 0s 2ms/step - loss: 2.4735e-08 - accuracy: 1.0000
Epoch 482/500
62/62 [==============================] - 0s 2ms/step - loss: 2.4372e-08 - accuracy: 1.0000
Epoch 483/500
62/62 [==============================] - 0s 2ms/step - loss: 2.4047e-08 - accuracy: 1.0000
Epoch 484/500
62/62 [==============================] - 0s 2ms/step - loss: 2.3709e-08 - accuracy: 1.0000
Epoch 485/500
62/62 [==============================] - 0s 2ms/step - 